<a href="https://colab.research.google.com/github/JaganK2Commit/Copilot/blob/main/BedTimeStory_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### What should be the video be about ?


In [2]:
TOPIC = "Could you please help me create a short story script that is perfect for entertaining a 5-year-old? The story should be magical, filled with colorful characters and have an educational message. Please make it fun, engaging, and age-appropriate." # make sure it has "how to _"
NARRATOR_ADJECTIVES = "story teller"
MUSIC_STYLE = "kids friendly random music"

### Install the dependencies

In [8]:
!pip install replicate
!pip install requests
!pip install openai
!pip install langchain
!pip install moviepy
!pip install ffmpeg --upgrade
!pip install pydub
from google.colab import output
output.clear()

In [9]:
import os
import openai
import time
import numpy as np
import replicate
import json

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, TransformChain
from langchain.llms import OpenAI, Replicate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from pydub import AudioSegment

## Aquire Tokens for Replicate and OpenAI

In [11]:
# get your token from https://replicate.com/account
from getpass import getpass

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


In [12]:
# get your key from https://platform.openai.com/account/api-keys
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

··········


## Create the LongChain

In [45]:
# LLMChain to write a a script for our how-to video.
topic_template = """
Generate a script for a short story for children of 5 to 7 years old, which should be in not more than 10 paragraphs.
Write the story in JSON format with the following keys:

"title": The name of the story
"description": Description that can be used for YouTube video description, not exceeding 20 words
"paragraphs": A list of 10 paragraphs that make up the story
"visual_descriptions": A list of 10 visual descriptions that correspond to each paragraph. Each visual description is an array object with two keys character_descriptions, scene_description. Each character_descriptions has two keys name and a description of the characters.

Please make sure that each visual description includes both the character introductions and the scene descriptions to allow AI image tools to generate appropriate pictures for each scene.

Thank you!
"""
system_message_prompt = SystemMessage(content="You are a story teller, shares wide range of fairy tales, bedtime stories, kids stories, and be creative in your responses")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=[]
                                                  ))

# create the initial script
chat = ChatOpenAI(temperature=0.99, model_name="gpt-3.5-turbo")
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
script_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='script')

script = script_chain.run({"topic": TOPIC, "narrator_adjectives": NARRATOR_ADJECTIVES, "music_style": MUSIC_STYLE})
# data = json.loads(result)
# script = data['paragraphs']
# video_descriptions = data['video_descriptions']

print(script)
print(chat_prompt_template)
# print(video_descriptions)

{
  "title": "The Brave Little Turtle",
  "description": "Join the brave little turtle on a thrilling adventure to save his underwater friends.",
  "paragraphs": [
    "Once upon a time, in the vast blue ocean, there lived a brave little turtle named Tommy.",
    "Tommy was always curious and loved exploring the ocean depths.",
    "One day, while swimming near a coral reef, Tommy discovered that his friends, the clownfish, were trapped in a fishing net.",
    "Tommy knew he had to help them despite being scared.",
    "With great courage, Tommy dived into action, biting through the net to free his friends.",
    "The clownfish were grateful and praised Tommy for his bravery.",
    "Word of Tommy's heroic act spread among the ocean creatures.",
    "Soon, a group of dolphins sought Tommy's help to rescue a lost baby dolphin.",
    "Tommy gathered his strength and fearlessly led the dolphins to find the little one.",
    "The baby dolphin was reunited with its family, and Tommy became a

In [57]:
script = """
{
    "title": "The Star Painter and The Moon",
    "description": "Embark on a magical adventure with Lila, the star painter, as she brings colors to the night sky and befriends the moon.",
    "paragraphs": [
        "Once upon a time, in a little cottage atop a hill, there lived a young girl named Lila who had the power to paint stars in the night sky.",
        "Every night, Lila would climb up the hill with her magical paintbrush and paint stars, making the night sky twinkle with colors.",
        "One night, as she painted, she noticed the moon looked sad. Lila decided to paint a picture of a flower next to the moon to cheer it up.",
        "To her surprise, the moon began to speak! It thanked Lila for the beautiful flower but shared that it felt lonely in the sky.",
        "Lila promised the moon that she would visit it every night. They started to share stories and soon became best friends.",
        "Lila wanted the moon to have more friends. So, she painted colorful planets and stars around the moon to keep it company.",
        "The moon was very happy and wanted to thank Lila. It used its magic to create a beautiful night-blooming flower and sent it down to her.",
        "With her magical paintbrush, Lila made the flower into a luminescent necklace that glowed with the colors of the night sky.",
        "As nights passed, the sky became a canvas of stories and friendship. Everyone from the village would gather to see the magical sky Lila and the moon created together.",
        "Lila never felt alone again, and neither did the moon. Their friendship was painted across the sky for everyone to see and filled the night with magic and wonder."
    ],
    "visual_descriptions": [
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress and holding a magical paintbrush."
                }
            ],
            "scene_descriptions": "Lila is standing outside a small cottage on a hill, looking up at the night sky filled with stars."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress, holding a magical paintbrush that glows."
                }
            ],
            "scene_descriptions": "Lila is on top of the hill painting stars in the night sky with her magical paintbrush."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress, looking concerned."
                }
            ],
            "scene_descriptions": "Lila is painting a flower next to a sad-looking moon in the night sky."
        },
        {
            "character_descriptions": [
                {
                    "name": "Moon",
                    "description": "Moon has a face and looks happy and surprised."
                }
            ],
            "scene_descriptions": "The moon starts to talk to Lila, who is holding her magical paintbrush."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress, smiling."
                },
                {
                    "name": "Moon",
                    "description": "Moon has a face and is smiling."
                }
            ],
            "scene_descriptions": "Lila and the moon are sharing stories, with colorful stars around them."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress, painting with her magical paintbrush."
                },
                {
                    "name": "Moon",
                    "description": "Moon has a face and looks happy with colorful planets and stars around it."
                }
            ],
            "scene_descriptions": "Lila is painting colorful planets and stars around the smiling moon."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress, looking amazed."
                },
                {
                    "name": "Moon",
                    "description": "Moon has a face and is smiling down at Lila."
                }
            ],
            "scene_descriptions": "The moon is sending down a night-blooming flower to Lila."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress and a glowing necklace."
                }
            ],
            "scene_descriptions": "Lila is turning the night-blooming flower into a glowing necklace with her magical paintbrush."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress and a glowing necklace, smiling."
                },
                {
                    "name": "Villagers",
                    "description": "Villagers are people of various ages looking up at the sky with awe."
                }
            ],
            "scene_descriptions": "Lila is standing on a hill with villagers around her, all looking up at the colorful night sky."
        },
        {
            "character_descriptions": [
                {
                    "name": "Lila",
                    "description": "Lila is a young girl with long blue hair, wearing a white dress and a glowing necklace, looking happy."
                },
                {
                    "name": "Moon",
                    "description": "Moon has a face and is smiling with colorful stars around it."
                }
            ],
            "scene_descriptions": "Lila is looking up at the happy moon in a night sky filled with colorful stars, planets, and paintings."
        }
    ]
}
"""

In [46]:
# LLMChain to write a title for our video
llm = OpenAI(temperature=.9)
template = """Please come up with a creative and zany title for the below how-to video script.
Puns are encouraged. Don't include quotations (") in the output.

Script:
{script}
Title: """
prompt_template = PromptTemplate(input_variables=["script"], template=template)
title_chain = LLMChain(llm=llm, prompt=prompt_template, output_key='title')

# title = title_chain.run({"script": script})
# print(title)



## Script Scrubbing

In [47]:
## Fill the characters_descriptions with the last known description for each paragraph

def fill_character_descriptions(input_data):
    character_descriptions = {}

    for item in input_data["visual_descriptions"]:
        current_character_descriptions = item.get("character_descriptions", [])
        for character_description in current_character_descriptions:
            character_name = character_description["name"]
            character_description_text = character_description["description"]
            character_descriptions[character_name] = character_description_text

    for item in input_data["visual_descriptions"]:
        updated_character_descriptions = []
        current_character_descriptions = item.get("character_descriptions", [])
        for character_description in current_character_descriptions:
            character_name = character_description["name"]
            if character_name in character_descriptions:
                updated_character_description = {
                    "name": character_name,
                    "description": character_descriptions[character_name]
                }
                updated_character_descriptions.append(updated_character_description)
        item["character_descriptions"] = updated_character_descriptions

    return input_data


# Fill the character_descriptions with the last defined value
script = fill_character_descriptions(json.loads(script))

# Print the result
# for item in filled_input_data['visual_descriptions']:
#     print(item)

In [96]:
# LLMChain to create the replicate predictions for our text-to-image model
import re
def update_scene_descriptions(input_data):
    character_descriptions = input_data["character_descriptions"]
    scene_description = input_data["scene_description"]

    updated_scene_descriptions = []

    if character_descriptions:
        for character_description in character_descriptions:
            character_name = character_description["name"]
            character_description_text = character_description["description"]

            # Check if the character name appears in the scene description
            if character_name in scene_description:
                # Append the character description to the scene description
                updated_scene_description = re.sub(re.escape(character_name), character_name + "," + character_description_text, scene_description, flags=re.IGNORECASE)
                updated_scene_descriptions.append(updated_scene_description)
            else:
              updated_scene_descriptions.append(scene_description)
    else:
        # If no character descriptions available, return original scene descriptions
        updated_scene_descriptions.append(scene_description)
    return updated_scene_descriptions


def transform_func(inputs: dict) -> dict:
  video_model = replicate.models.get('ai-forever/kandinsky-2')
  video_version = video_model.versions.get("601eea49d49003e6ea75a11527209c4f510a93e2112c969d548fbb45b9c4f19f")
  descriptions = inputs['script']['visual_descriptions']

  predictions = []
  known_character_descriptions = [];

  for description in descriptions:
      print(description)
      updated_description = update_scene_descriptions(description)[0]
      print(f"Creating video prediction for '{updated_description}'...")
      video_prediction = replicate.predictions.create(version=video_version,
                                                      input={"prompt": updated_description, "prior_steps": '5', "guidance_scale": 4, "num_inference_steps": 100, "prior_cf_scale":4,
                                                             "scheduler": "p_sampler", "width": 1024, "height":768})
      predictions.append(video_prediction)
  return {'video_predictions': predictions}

video_predictions_chain = TransformChain(input_variables=['script'], output_variables=['video_predictions'], transform=transform_func)

# video_predictions = video_predictions_chain.run({"script": script})
# print(video_predictions)

In [97]:
# LLMChain to create the replicate predictions for our bark model
def transform_func(inputs: dict) -> dict:
  audio_model = replicate.models.get("suno-ai/bark")
  audio_version = audio_model.versions.get("b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787")
  parsed_script = inputs['script']['paragraphs']

  predictions = []

  for line in parsed_script:
      print(f"Creating audio prediction for '{line}''...")
      audio_prediction = replicate.predictions.create(version=audio_version,
                                                      input={"prompt": line, "history_prompt": "announcer", "text_temp": 0.7, "waveform_temp":0.8})
      predictions.append(audio_prediction)
  return {'audio_predictions': predictions}

audio_predictions_chain = TransformChain(input_variables=['script'], output_variables=['audio_predictions'], transform=transform_func)

# audio_predictions = audio_predictions_chain.run({"script": script})
# print(audio_predictions)

In [98]:
# LLMChain to create the cover image
llm = OpenAI(temperature=.9)
template = """
          Create a visual description artstation, hd, dramatic lighting, detailed for the following script.
          "{script}""
          """
prompt_template = PromptTemplate(input_variables=["script"], template=template)

text2image = Replicate(model="ai-forever/kandinsky-2:601eea49d49003e6ea75a11527209c4f510a93e2112c969d548fbb45b9c4f19f",
                       input={"prior_steps": '5', "guidance_scale": 4, "num_inference_steps": 100, "prior_cf_scale":4,
                                                        "scheduler": "p_sampler"})

# text2image = Replicate(model="stability-ai/stable-diffusion:db21e45d3f7023abc2a46ee38a23973f6dce16bb082a930b0c49861f96d1e5bf",
#                        input={'image_dimensions': '512x512', "negative_prompt": "text, writing"})
title_image_chain = LLMChain(llm=text2image, prompt=prompt_template, output_key='title_image')

# title_image = title_image_chain.run({"script": script})
# print(title_image)

In [99]:
# LLMChain to write the thank you note at the end of our video
template = """Please come up with a creative and zany ending quote from our narrator.
The script is what the narrator just read. We want to close things out in less than 30 words.

Make sure you add a "And don't forget to like and subscribe!" to the end of your output.

You are a {narrator_adjectives} narrator.

Script:
{script}
Ending quote:
"""

system_message_prompt = SystemMessage(content="You are a helpful assistant.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=template,
                                                  input_variables=["script", "narrator_adjectives"]))
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
ending_quote_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='ending_quote')

# ending_quote = ending_quote_chain.run({"script": script, "narrator_adjectives": NARRATOR_ADJECTIVES})
# print(ending_quote)

In [100]:
# LLMChain to create the prediction that generates the audio for the thank you note
def transform_func(inputs: dict) -> dict:
  audio_model = replicate.models.get("suno-ai/bark")
  audio_version = audio_model.versions.get("b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787")
  ending_quote_prediction = replicate.predictions.create(version=audio_version,
                                                      input={"prompt": inputs['ending_quote'], "history_prompt": "announcer", "text_temp": 0.7, "waveform_temp":0.8}) #en_speaker_6 female, loud
  return {'ending_quote_prediction': ending_quote_prediction}

ending_quote_prediction_chain = TransformChain(input_variables=['ending_quote'], output_variables=['ending_quote_prediction'], transform=transform_func)

# ending_quote_prediction = ending_quote_prediction_chain.run({"ending_quote": ending_quote})
# print(ending_quote_prediction)

#chain_output['ending_quote_prediction'].output['audio_out']


In [101]:
# LLMChain to create the prediction that generates the audio for the title
def transform_func(inputs: dict) -> dict:
  audio_model = replicate.models.get("suno-ai/bark")
  audio_version = audio_model.versions.get("b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787")
  title_prediction = replicate.predictions.create(version=audio_version,
                                                      input={"prompt": inputs['title'],"history_prompt": "announcer", "text_temp": 0.7, "waveform_temp":0.8})
  return {'title_audio_prediction': title_prediction}

title_audio_prediction_chain = TransformChain(input_variables=['title'], output_variables=['title_audio_prediction'], transform=transform_func)

# title_audio_prediction = title_audio_prediction_chain.run({"title": title})
# print(title_audio_prediction)


In [102]:
# LLMChain to create the prediction that generates the background music
def transform_func(inputs: dict) -> dict:
  model = replicate.models.get("riffusion/riffusion")
  version = model.versions.get("8cf61ea6c56afd61d8f5b9ffd14d7c216c0a93844ce2d82ac1c9ecc9c7f24e05")
  music_prediction = replicate.predictions.create(version=version, input={"prompt": inputs['music_style']})

  return {'music_prediction': music_prediction}

music_prediction_chain = TransformChain(input_variables=['music_style'], output_variables=['music_prediction'], transform=transform_func)

music_prediction = music_prediction_chain.run({"music_style": MUSIC_STYLE})
print(music_prediction)

id='d2vvlojbnqejemqfp7khbth7va' error=None input={'prompt': 'kids friendly random music'} logs='' output=None status='starting' version=Version(id='8cf61ea6c56afd61d8f5b9ffd14d7c216c0a93844ce2d82ac1c9ecc9c7f24e05', created_at=datetime.datetime(2022, 12, 16, 7, 48, 40, 890140, tzinfo=datetime.timezone.utc), cog_version='0.6.1', openapi_schema={'info': {'title': 'Cog', 'version': '0.1.0'}, 'paths': {'/': {'get': {'summary': 'Root', 'responses': {'200': {'content': {'application/json': {'schema': {}}}, 'description': 'Successful Response'}}, 'operationId': 'root__get'}}, '/predictions': {'post': {'summary': 'Predict', 'responses': {'200': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/PredictionResponse'}}}, 'description': 'Successful Response'}, '422': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'}}}, 'description': 'Validation Error'}}, 'description': 'Run a single prediction on the model', 'operationId': 'pre

## Run the chain

Now, let's execute the chain we created. This is relatively fast, because the chains that create long-running predictions (like the video_predictions_chain) make asynchronous calls to the Replicate API.

In [ ]:
# Run the chain
overall_chain = SequentialChain(chains=[
    #script_chain,
                                        title_chain,
                                        video_predictions_chain,
                                        audio_predictions_chain,
                                        ending_quote_chain,
                                        ending_quote_prediction_chain,
                                        title_image_chain,
                                        title_audio_prediction_chain,
                                        music_prediction_chain
                                        ], input_variables=['script', 'topic', 'narrator_adjectives', 'music_style'], output_variables=[
                                            #'script',
                                            'title', 'video_predictions', 'audio_predictions', 'ending_quote', 'title_image', 'ending_quote_prediction', 'title_audio_prediction', 'music_prediction'], verbose=True)
chain_output = overall_chain({"script":script, "topic": TOPIC, "narrator_adjectives": NARRATOR_ADJECTIVES, "music_style": MUSIC_STYLE})



> Entering new  chain...
{'character_descriptions': [{'name': 'Tommy', 'description': 'A small green turtle with a proud expression.'}], 'scene_description': 'An endless blue ocean with colorful coral reefs.'}
Creating video prediction for 'An endless blue ocean with colorful coral reefs.'...
{'character_descriptions': [{'name': 'Tommy', 'description': 'A small green turtle with a proud expression.'}], 'scene_description': 'Tommy exploring the ocean depths with sunlight filtering through the water.'}
Creating video prediction for 'Tommy,A small green turtle with a proud expression. exploring the ocean depths with sunlight filtering through the water.'...
{'character_descriptions': [{'name': 'Tommy', 'description': 'A small green turtle with a proud expression.'}, {'name': 'Clownfish', 'description': 'Bright orange fish with white stripes, trapped in a fishing net.'}], 'scene_description': 'Tommy finding his trapped friends, surrounded by colorful coral and the shimmering net.'}
Creat

In [ ]:
# unpack outputs
script = chain_output['script']
title = script['title']
split_script = script['paragraphs']
video_descriptions = script['visual_descriptions']
video_predictions = chain_output['video_predictions']
audio_predictions = chain_output['audio_predictions']

# print(title)
# print(split_script)
# print(video_descriptions)
# print(video_predictions)
# print(audio_predictions)

In [ ]:
# sanity check
assert len(split_script) == len(video_descriptions)

# ⏳ Wait for our async predictions to complete
Here's a helper to check in on our predictions. This usually takes a minute or two.

In [ ]:
def all_done(predictions):
    return set([p.status for p in predictions]) == {'succeeded'}

In [ ]:
all_predictions = chain_output['video_predictions'] + \
                  chain_output['audio_predictions'] + \
                  [chain_output['ending_quote_prediction']] + \
                  [chain_output['title_audio_prediction']] + \
                  [chain_output['music_prediction']]

In [ ]:
done = False

while not done:
  [p.reload() for p in all_predictions]
  for p in all_predictions:
    print(f'https://replicate.com/p/{p.id}', p.status)
  done = all_done(all_predictions)
  time.sleep(2)
  output.clear()

print("Predictions complete")

# 🪡 Stitch them all together!

In [ ]:
video_urls = [v.output for v in video_predictions]
audio_urls = [a.output['audio_out'] for a in audio_predictions]
music_url = chain_output['music_prediction'].output['audio']
subtitles = split_script
title_image_url = chain_output['title_image']
title_audio_url = chain_output['title_audio_prediction'].output['audio_out']

video_urls


In [ ]:
## slow down the audio urls

import requests
import os

def slow_down_audio(input_file, output_file, slowdown_factor):
    # Load the audio file
    audio = AudioSegment.from_wav(input_file)

    # Calculate the new length of the audio
    new_length = int(len(audio) / slowdown_factor)

    # Stretch the audio by duplicating frames
    slowed_audio = audio._spawn(audio.raw_data, overrides={
        "frame_rate": int(audio.frame_rate * slowdown_factor)
    })

    # Export the modified audio to a file
    slowed_audio.export(output_file, format="mp3")
    print("Slowdown complete. Output file:", output_file)

slowdown_factor = 0.96

In [ ]:
audio_files = []

for i, url in enumerate(audio_urls):
    response = requests.get(url)
    audio_filename = f"temp_audio{i}.wav"
    with open(audio_filename, "wb") as audio_file:
        audio_file.write(response.content)

    # Generate the output file name
    output_file = os.path.join(audio_filename.replace(".wav", ".mp3"))

    # Call the slow_down_audio function
    slow_down_audio(audio_filename, output_file, slowdown_factor)

    # Append the output file name to the array
    audio_files.append(output_file)
    #audio_files.append(audio_filename)

In [ ]:
import requests
import os
import moviepy.editor as mp
import moviepy.video.fx.all as vfx
import textwrap
from moviepy.editor import *
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import numpy as np


# Download video and audio files
video_files = []
audio_files = []
# for i, url in enumerate(video_urls):
#     response = requests.get(url)
#     video_filename = f"temp_video{i}.mp4"
#     with open(video_filename, "wb") as video_file:
#         video_file.write(response.content)
#     video_files.append(video_filename)
fps = 12.0
for i, url in enumerate(video_urls):
    response = requests.get(url[0])
    image = Image.open(BytesIO(response.content))
    image_np = np.array(image)
    clip = mp.ImageSequenceClip([image_np], fps=fps)
    video_filename = f"temp_video{i}.mp4"
    clip.write_videofile(video_filename, codec='libx264', fps=fps)
    #with open(video_filename, "wb") as video_file:
        #video_file.write(response.content)
    video_files.append(video_filename)

for i, url in enumerate(audio_urls):
    response = requests.get(url)
    audio_filename = f"temp_audio{i}.wav"
    with open(audio_filename, "wb") as audio_file:
        audio_file.write(response.content)

    # Generate the output file name
    output_file = os.path.join(audio_filename.replace(".wav", ".mp3"))

    # Call the slow_down_audio function
    slow_down_audio(audio_filename, output_file, slowdown_factor)

    # Append the output file name to the array
    audio_files.append(output_file)
    #audio_files.append(audio_filename)

# Load and process video and audio files

processed_videos = []
for i, audio_file in enumerate(audio_files):
    video = mp.VideoFileClip(video_files[i])
    audio = mp.AudioFileClip(audio_file)

    # Loop the video for the duration of the audio
    looped_video = mp.concatenate_videoclips([video] * int(audio.duration // video.duration + 1))

    # Set the audio of the video to the audio file
    video_with_audio = looped_video.set_audio(audio)
    processed_videos.append(video_with_audio)

# Concatenate all the processed videos
final_video = mp.concatenate_videoclips(processed_videos)

## The following adds the title image / narration to the video.
# Add this function to create the text image
def txt_image(img, txt, font_size, color):
    image = img.copy()
    draw = ImageDraw.Draw(image)
    draw.text((50, 50), txt, fill=(255, 255, 0))
    # font = ImageFont.load_default().font_variant(size=font_size)
    # draw.text((50, 50), txt, font=font, fill=color)
    return image

# Download and create the image clip
image_url = title_image_url
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Resize the image to match the video dimensions
img_resized = img.resize((1200, 900))

# Download the audio file
audio_url = chain_output['ending_quote_prediction'].output['audio_out']
response = requests.get(audio_url)
audio_filename = "temp_audio_ending.wav"
with open(audio_filename, "wb") as audio_file:
    audio_file.write(response.content)

# Generate the output file name
output_file = os.path.join(audio_filename.replace(".wav", ".mp3"))

# Call the slow_down_audio function
slow_down_audio(audio_filename, output_file, slowdown_factor)

# Create the audio clip
audio_ending = AudioFileClip(output_file)

# make title empty for now, couldn't figure out how to get it bigger
text = chain_output['title']
img_text = ImageClip(np.asarray(txt_image(img_resized, txt='text', font_size=72, color="white")), duration=4)

# Set the audio of the image clip to the audio file and trim it to the same duration
img_text_audio_ending = mp.concatenate_videoclips([img_text] * int(audio_ending.duration // img_text.duration + 1))
img_text_audio_ending = img_text.set_audio(audio_ending)

# Download the title page audio file
audio_url = chain_output['title_audio_prediction'].output['audio_out']
response = requests.get(audio_url)
with open("temp_audio_title.mp3", "wb") as audio_file:
    audio_file.write(response.content)

# Create the audio clip
audio_beginning = AudioFileClip("temp_audio_title.mp3")

# Set the audio of the image clip to the audio file and trim it to the same duration
img_text_audio_beginning = mp.concatenate_videoclips([img_text] * int(audio_beginning.duration // img_text.duration + 1))
img_text_audio_beginning = img_text.set_audio(audio_beginning)

# Concatenate the image clip with the processed videos
width, height = processed_videos[0].size
title_video = img_text_audio_beginning.resize((width, height))
ending_video = img_text_audio_ending.resize((width, height))

# processed_videos.insert(0, title_video)
processed_videos.append(ending_video)

final_video = concatenate_videoclips(processed_videos)

# Download the background audio file
bg_audio_url = music_url
response = requests.get(bg_audio_url)
with open("temp_bg_audio.mp3", "wb") as audio_file:
    audio_file.write(response.content)

# Create the background audio clip
bg_audio = AudioFileClip("temp_bg_audio.mp3")

# Calculate the duration of the final video
video_duration = final_video.duration

# Loop the background audio to match the final video's duration
bg_audio_looped = bg_audio.fx(afx.audio_loop, duration=video_duration)
bg_audio_looped = bg_audio_looped.volumex(0.3)

# Overlay the background audio with the audio from the final video
final_audio = CompositeAudioClip([final_video.audio, bg_audio_looped])

# Set the audio of the final video to the combined audio
final_video_with_bg_audio = final_video.set_audio(final_audio)

# Save the final video
final_video_with_bg_audio.write_videofile(f"how_to.mp4", codec='libx264', audio_codec='aac')

# Clean up temporary files
for video_file, audio_file in zip(video_files, audio_files):
    os.remove(video_file)
# for audio_file in audio_files:
#     os.remove(audio_file)

In [ ]:
#@title Watch the video
from IPython.display import HTML
from base64 import b64encode
mp4 = open(f'how_to.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
chain_output['ending_quote_prediction'].output['audio_out']